In [2]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
from scipy.stats import norm, skew
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import matplotlib

pd.set_option('max_columns',1000)
pd.set_option('max_rows',1000)

import warnings
warnings.filterwarnings('ignore')

import re
import geocoder
from geopy.distance import great_circle, vincenty
from tqdm import tqdm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import os
import gc
import lightgbm as lgb
from sklearn.model_selection import KFold, train_test_split
from time import time
import datetime
from script import RegressionPredictor,LogRegressionPredictor,plot_scatter
import japanize_matplotlib
# print(os.listdir("././input"))
# print(os.listdir("././submit"))
from utils import save_data
SEED=1234
n_splits=10
import glob

X_train=pd.read_csv('./psuedo/X_train1107.csv').sample(200)
X_test=pd.read_csv('./psuedo/X_test1107.csv').sample(200)
y_train = X_train['賃料']
drop_col = ['id','賃料']
y_train = X_train['賃料']

y_train_log = np.log1p(y_train)
X_train = X_train.drop(drop_col,axis=1)
X_test = X_test.drop('id',axis=1)

for i in range(100):
    folder=KFold(n_splits=10, shuffle=True, random_state=i)
    lgbm_params = {
        'num_iterations': 50000,
        'learning_rate': 0.01,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'reg_lambda': 68.19849585036235,
        'reg_alpha': 0.007585940082730346,
        'colsample_bytree': 0.6,
        'subsample': 0.5,
        'max_depth': 8,
        'min_child_weight': 8,
        'lambda': 1,
        'eta': 0.1,
        'verbose': 1,
        'random_state': i,
        'early_stopping_round': 100
        }

    lgbm_log_params=lgbm_params.copy()
    lgbm_log_params['metric']='mae'
    lgbm_log_params['learning_rate']=0.05

    lgbm_x_params=lgbm_params.copy()
    lgbm_x_params['metric']='cross_entropy'
    lgbm_x_params['learning_rate']=0.05
    
    LogLGBM = LogRegressionPredictor(X_train, (y_train/X_train['面積']).values, X_test, Folder=folder, params=lgbm_log_params,
                                      sk_model=None, n_splits=10, clf_type='lgb')
    log_lgboof, log_lgbpreds, log_lgbFIs = LogLGBM.fit()
    save_data(X_train,log_lgboof, log_lgbpreds,rmse=LogLGBM.rmse(),name=f'log_tika_{i}',save_dir='./seed_average')